In [1]:
import requests
from bs4 import BeautifulSoup
from IPython.display import HTML, display
import re
import html

Код собирает ссылки на все выпуски из архива

In [3]:
arch = []
for page in range(1,24):
    reqlink = 'http://sprostor.ru/?module=arhiveis&action=list&id=0&page=' + str(page)
    req = requests.get(reqlink)
    soup = BeautifulSoup(re.sub('&amp;', '&', req.text, flags=re.DOTALL), 'lxml')
    for i in soup.findAll('div', attrs = {'class':'archiveis_item'}):
        href = re.sub('amp;', '', re.findall('<a href=\"([^\"]*)\">№', str(i), flags=re.DOTALL)[0])
        arch.append('http://sprostor.ru/' + href)

Собирает ссылки на статьи

In [4]:
articles = []
for a in arch:
    page = re.sub('&amp;', '&', requests.get(a).text, flags=re.DOTALL)
    articles += re.findall('<a href=\'([^\']*)\' class=\'link_intro\'>', page, flags=re.DOTALL)

In [15]:
articles = list(set(articles))

Выкачивает статьи и записывает в файлы

In [59]:
cnt = 0
for a in articles:
    cnt += 1
    NAME = str(cnt) + '.txt'
    url = 'http://sprostor.ru/' + a
    page = html.unescape(requests.get(url).text)
    try:
        meta = re.findall('<meta name=\"keywords" content=\"([^\"]*)\".*?class=\'date_start\'>([^\<]*)</span>', page, flags=re.DOTALL)[0]
    #@ti @da @au
        au = re.findall('<strong>([^\<]*)</strong></p></span><span class=\'author\'>', page, flags=re.DOTALL)
        if au == []:
            aut = 'Noname'
        else:
            aut = au[0]
        page = re.sub('</?strong>','', page, flags=re.DOTALL)
        page = re.sub('</p>','\n', page, flags=re.DOTALL)
        page = re.sub('<p>','', page, flags=re.DOTALL)
        page = re.sub('</?em>','', page, flags=re.DOTALL)
        tekst = re.sub('<.*?>','', re.findall('<span class=\'text\'>(.*?)</span><span class=\'author\'>', page, flags=re.DOTALL)[0], flags=re.DOTALL)
        date = '.'.join(meta[1].split('-')[::-1])
        file = '@ti ' + meta[0] + '\n@da ' + date + '\n@au ' + aut + '\n@url ' + url + '\n' + tekst
        f = open(NAME, 'w', encoding='utf-8')
        f.write(file)
        f.close()
    except:
        print(url)

http://sprostor.ru/?module=articles&action=view&id=3174
